# Binarized Convolutional Neural Networks

This tutorial demonstrates training a simple Binarized Convolutional Neural Network to classify MNIST digits. This simple network will achieve over 98% accuracy on the MNIST test set. Because this tutorial uses XQuant and the [Keras Sequential API](https://www.tensorflow.org/guide/keras), creating and training our model will take just a few lines of code.

### Import TensorFlow and XQuant

In [1]:
import tensorflow as tf
import xquant as xq

### Download and prepare the MNIST dataset

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

### Create the model

This will create a simple binarized convolutional network trained using the straight-through estimator (STE)

In [4]:
kwargs = dict(input_quantizer="ste_sign", kernel_quantizer="ste_sign", kernel_constraint="weight_clip")

model = tf.keras.models.Sequential()

model.add(xq.layers.QuantConv2D(32, (3, 3), kernel_quantizer="ste_sign", kernel_constraint="weight_clip", use_bias=False, input_shape=(28, 28, 1)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(tf.keras.layers.Activation("hard_tanh"))

model.add(xq.layers.QuantConv2D(64, (3, 3), use_bias=False, **kwargs))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(tf.keras.layers.Activation("hard_tanh"))

model.add(xq.layers.QuantConv2D(64, (3, 3), use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(tf.keras.layers.Activation("hard_tanh"))
model.add(tf.keras.layers.Flatten())

model.add(xq.layers.QuantDense(10, use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(tf.keras.layers.Activation("softmax"))

 Here's the complete architecture of our model.
 
 Almost all parameter of the network a binarized (either -1 or 1). This will make the network extremly fast when deployed on a embedded device that supports binarized neural networks.

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quant_conv2d_3 (QuantConv2D) (None, 26, 26, 32)        288       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
batch_normalization_v1_4 (Ba (None, 13, 13, 32)        96        
_________________________________________________________________
activation_4 (Activation)    (None, 13, 13, 32)        0         
_________________________________________________________________
quant_conv2d_4 (QuantConv2D) (None, 11, 11, 64)        18432     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
batch_normalization_v1_5 (Ba (None, 5, 5, 64)          192       
__________

### Compile and train the model

Note: This may take a few minutes depending on your system.

In [8]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, batch_size=64, epochs=6)

Epoch 1/6
60000/60000 [==============================] - 79s 1ms/sample - loss: 0.5970 - acc: 0.9440
Epoch 2/6
60000/60000 [==============================] - 81s 1ms/sample - loss: 0.5153 - acc: 0.9645
Epoch 3/6
60000/60000 [==============================] - 76s 1ms/sample - loss: 0.4976 - acc: 0.9693
Epoch 4/6
60000/60000 [==============================] - 77s 1ms/sample - loss: 0.4903 - acc: 0.9711
Epoch 5/6
60000/60000 [==============================] - 80s 1ms/sample - loss: 0.4840 - acc: 0.9724
Epoch 6/6
60000/60000 [==============================] - 76s 1ms/sample - loss: 0.4818 - acc: 0

### Evaluate the model

In [11]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

10000/10000 [==============================] - 3s 290us/sample - loss: 0.4339 - acc: 0.9806


In [10]:
print(f"Test accuracy {test_acc * 100:.2f} %")

Test accuracy 98.06 %


As you can see, our simple binarized CNN has achieved a test accuracy of over 98 %. Not bad for a few lines of code!